# Camera UI

In [1]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [2]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

# Code

In [5]:
from IPython.display import Image
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np
import PIL.Image
import random
from IPython.display import Image
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

answer = input("Do you want to play rock paper scissors?(yes/no) ")
answer = answer.lower()
if answer=="no":
  print("Thanks for playing!")

while answer=="yes":
  c=0
  u=0
  while c<2 and u<2:
    print("This is a game for the best out of 3, aka first one to 2 points!")
    print("Now select rock, paper, or scissors: ")
    try:
      filename = take_photo()
      print('Saved to {}'.format(filename))

      # Show the image which was just taken.
      display(Image(filename))
    except Exception as err:
      # Errors will be thrown if the user does not have a webcam or if they do not
      # grant the page permission to access it.
      print(str(err))

    if not hasattr(PIL.Image, 'Resampling'):  # Pillow<9.0
      PIL.Image.Resampling = PIL.Image
    # Disable scientific notation for clarity
    np.set_printoptions(suppress=True)

    # Load the model
    model = load_model("keras_model.h5", compile=False)

    # Load the labels
    class_names = open("labels.txt", "r").readlines()

    # Create the array of the right shape to feed into the keras model
    # The 'length' or number of images you can put into the array is
    # determined by the first position in the shape tuple, in this case 1
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # Replace this with the path to your image
    image = Image.open("photo.jpg").convert("RGB")

    # resizing the image to be at least 224x224 and then cropping from the center
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

    # turn the image into a numpy array
    image_array = np.asarray(image)

    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Load the image into the array
    data[0] = normalized_image_array

    # Predicts the model
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    # Print prediction and confidence score
    print("This is what you chose:", class_name, end="")
    print("Confidence Score:", confidence_score)
    possible_actions = ["rock", "paper", "scissors"]
    computer_action = random.choice(possible_actions)
    print("This is what the computer chooses:",computer_action.capitalize())
    outcome = ""

    if computer_action.capitalize() == "Paper" and str.__contains__(class_name, "Paper"):
      outcome = "We tied"
      print(outcome)
    elif computer_action.capitalize() == "Rock" and str.__contains__(class_name, "Rock"):
      outcome = "We tied"
      print(outcome)
    elif computer_action.capitalize() == "Scissors" and str.__contains__(class_name, "Scissors"):
      outcome = "We tied"
      print(outcome)
    elif computer_action == "rock" and str.__contains__(class_name, "Paper"):
      outcome = "You win"
      print(outcome)
      u+=1
    elif computer_action == "rock" and str.__contains__(class_name, "Scissors"):
      outcome = "Haha, computer wins!"
      print(outcome)
      c+=1
    elif computer_action == "paper" and str.__contains__(class_name, "Rock"):
      outcome = "Haha, computer wins!"
      print(outcome)
      c+=1
    elif computer_action == "paper" and str.__contains__(class_name, "Scissors"):
      outcome = "You win"
      print(outcome)
      u+=1
    elif computer_action == "scissors" and str.__contains__(class_name, "Paper"):
      outcome = "Haha, computer wins!"
      print(outcome)
      c+=1
    elif computer_action == "scissors" and str.__contains__(class_name, "Rock"):
      outcome = "You win"
      print(outcome)
      u+=1
    print("Computer score: ",c)
    print("Your score: ",u)
    if u==2 or c==2:
      break
  # Disable scientific notation for clarity
  print("Let me get a reaction to the outcome!")
  try:
    filename = take_photo()
    print('Saved to {}'.format(filename))

    # Show the image which was just taken.
    display(Image(filename))
  except Exception as err:
    # Errors will be thrown if the user does not have a webcam or if they do not
    # grant the page permission to access it.
    print(str(err))
  np.set_printoptions(suppress=True)

    # Load the model
  model = load_model("keras_model1.h5", compile=False)

    # Load the labels
  class_names = open("labels1.txt", "r").readlines()

    # Create the array of the right shape to feed into the keras model
    # The 'length' or number of images you can put into the array is
    # determined by the first position in the shape tuple, in this case 1
  data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

    # Replace this with the path to your image
  image = Image.open("photo.jpg").convert("RGB")

    # resizing the image to be at least 224x224 and then cropping from the center
  size = (224, 224)
  image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

    # turn the image into a numpy array
  image_array = np.asarray(image)

    # Normalize the image
  normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

    # Load the image into the array
  data[0] = normalized_image_array

    # Predicts the model
  prediction = model.predict(data)
  index = np.argmax(prediction)
  class_name = class_names[index]
  confidence_score = prediction[0][index]

    # Print prediction and confidence score

  if u==2:
     print("Because you won, you seem", class_name.lower(),end="")
     print("Confidence Score:", confidence_score)
  elif c==2:
    print("Because you lost, you seem", class_name.lower(),end="")
    print("Confidence Score:", confidence_score)
  answer = input("Do you want to play rock paper scissors again?(yes/no) ")
  answer = answer.lower()




print("Thanks for playing!")






Do you want to play rock paper scissors?(yes/no) yes
This is a game for the best out of 3, aka first one to 2 points!
Now select rock, paper, or scissors: 


<IPython.core.display.Javascript object>

Saved to photo.jpg
'module' object is not callable
1/1 [==============================] - 1s 663ms/step
This is what you chose: Paper
Confidence Score: 0.9999919
This is what the computer chooses: Paper
We tied
Computer score:  0
Your score:  0
This is a game for the best out of 3, aka first one to 2 points!
Now select rock, paper, or scissors: 


<IPython.core.display.Javascript object>

Saved to photo.jpg
'module' object is not callable
1/1 [==============================] - 1s 687ms/step
This is what you chose: Scissors
Confidence Score: 0.9991885
This is what the computer chooses: Scissors
We tied
Computer score:  0
Your score:  0
This is a game for the best out of 3, aka first one to 2 points!
Now select rock, paper, or scissors: 


...

Saved to photo.jpg
'module' object is not callable
1/1 [==============================] - 1s 631ms/step
This is what you chose: Rock
Confidence Score: 0.9998344
This is what the computer chooses: Paper
Haha, computer wins!
Computer score:  1
Your score:  1
This is a game for the best out of 3, aka first one to 2 points!
Now select rock, paper, or scissors: 


<IPython.core.display.Javascript object>

Saved to photo.jpg
'module' object is not callable
1/1 [==============================] - 1s 775ms/step
This is what you chose: Rock
Confidence Score: 0.99534804
This is what the computer chooses: Scissors
You win
Computer score:  1
Your score:  2
Let me get a reaction to the outcome!


<IPython.core.display.Javascript object>

Saved to photo.jpg
'module' object is not callable
1/1 [==============================] - 1s 673ms/step
Because you won, you seem angry
Confidence Score: 0.77885956
Do you want to play rock paper scissors again?(yes/no) no
Thanks for playing!
